In [1]:
import pymssql
import pandas as pd
import numpy as np

In [6]:
def dataframe_from_query(given_cursor, given_query):
    given_cursor.execute(given_query)
    return_data = given_cursor.fetchall()
    dfr_columns = [item[0] for item in given_cursor.description]
    return_dataframe = pd.DataFrame(data=return_data, columns=dfr_columns)
    
    return return_dataframe

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
conn = pymssql.connect(
    server='10.100.200.3',
    port='1433',
    user='pawel', 
    password='20Al3Mot@VP@weL22#',
    database='SPEED',
    charset='ISO-8859-2')
cursor = conn.cursor()

In [14]:
zlec_query = """ SELECT 
[ID_ZLECENIA] 
,[NR_ZLECENIA] 
,[SPEDYTOR]
,[OPIEKUN]
,[ZA_MIEJSCE] 
,[ZA_MIASTO] 
,[ZA_KRAJ] 
,[ZA_KOD] 
,[TRASA]
,[ZA_DATA]
,[ZA_DATA_RZ]
,[WY_DATA]
,[WY_DATA_RZ]
,[WY_DATA_RZ_K]
,[WY_MIEJSCE] 
,[WY_MIASTO] 
,[WY_KRAJ] 
,[WY_KOD] 
,[KWOTA_K] 
,[WALUTA_K] 
,[FAKTURA] 
,[FAKTURA_ZB_ID] 
,[FAKTURA_ZB]  
,[NR_ZLECENIA_K] 
,[KWOTA_P] 
,[WALUTA_P] 
,[WALUTA_FV_P] 
,[FAKTURA_K] 
,[FAKTURA_K_ZB] 
,[FAKTURA_K_ZB_ID]
,[LOKALIZACJA]
FROM [SPEED].[dbo].[ZLECENIA] 
WHERE LOKALIZACJA = 'KAT_MS' AND MONTH([ZA_DATA]) = 1 AND YEAR([ZA_DATA]) = 2022 AND [ZAFAKTUROWANE] = 1
"""

zlecenia_query = """ SELECT 
*
FROM [SPEED].[dbo].[ZLECENIA] 
WHERE LOKALIZACJA = 'KAT_MS' AND MONTH([ZA_DATA]) = 1 AND YEAR([ZA_DATA]) = 2022 AND [ZAFAKTUROWANE] = 1
"""

zlec_df = dataframe_from_query(cursor, zlec_query)


zlec_df.FAKTURA_K_ZB_ID = zlec_df.FAKTURA_K_ZB_ID.astype(str).replace('nan', '0')
zlec_df.FAKTURA_K_ZB_ID = zlec_df.FAKTURA_K_ZB_ID.astype(float).astype(int)
zlec_df.FAKTURA_K_ZB_ID = zlec_df.FAKTURA_K_ZB_ID.astype(str).replace('0', '')
zlec_df.fillna('', inplace=True)
zlec_df.FAKTURA_ZB_ID = zlec_df.FAKTURA_ZB_ID.astype(str).replace('0', '')

for col in zlec_df.columns:
    if isinstance(zlec_df[col].iloc[1], str):
        try:
            zlec_df[col] = zlec_df[col].str.replace('Ľ', 'Ą')
        except ValueError as excc:
            print(col, excc)
            pass

# for col in zlec_df.columns:
#     if isinstance(zlec_df[col].iloc[1], str):
#         try:
#             zlec_df[col] = zlec_df[col].decode('ISO-8859-2')
#         except ValueError as excc:
#             print(col, excc)
#             pass

AttributeError: 'Series' object has no attribute 'decode'

In [ ]:
zlec_cols = ['ID_ZLECENIA', 'KWOTA_K', 'WALUTA_K', 'FAKTURA', 'FAKTURA_ZB_ID', 'FAKTURA_ZB', 'NR_ZLECENIA_K', 
             'KWOTA_P', 'WALUTA_P', 'FAKTURA_K', 'FAKTURA_K_ZB', 'FAKTURA_K_ZB_ID']

In [ ]:
zlec_df.fillna('', inplace=True)

In [15]:
data_cols = [el for el in zlec_df.columns if 'DATA' in el]

In [16]:
zlec_df[zlec_cols + ['OPIEKUN', 'SPEDYTOR'] + data_cols].sample(10)

,ID_ZLECENIA,KWOTA_K,WALUTA_K,FAKTURA,FAKTURA_ZB_ID,FAKTURA_ZB,NR_ZLECENIA_K,KWOTA_P,WALUTA_P,FAKTURA_K,FAKTURA_K_ZB,FAKTURA_K_ZB_ID,OPIEKUN,SPEDYTOR,ZA_DATA,ZA_DATA_RZ,WY_DATA,WY_DATA_RZ,WY_DATA_RZ_K
185,8072,1200.0,EUR,,13128,00094/FV/01/2022/MS,926,1050.0,EUR,160/36/01/2022,,,ARTUR FRĽSZCZAK,NATALIA KANAS,2022-01-14,2022-01-14 00:00:00,2022-01-17,2022-01-17,2022-01-17
330,8262,1580.0,PLN,00114/FV/01/2022/MS,,,FR/2022/01/20/2EVQBE,280.0,EUR,50/2022,,,OSKAR RYCZYŃSKI,OSKAR RYCZYŃSKI,2022-01-22,,2022-01-24,2022-01-24,2022-01-24
96,7954,850.0,PLN,,13006,00046/FV/01/2022/MS,PT/2022/152-05.01/05.01.2021,750.0,PLN,,1/01/2022,12931,JAKUB GAWLICZEK,GRZEGORZ KACZAN,2022-01-05,,2022-01-05,2022-01-05,2022-01-05
138,8015,1100.0,PLN,,13006,00046/FV/01/2022/MS,PT/2022/161-10.01/10.01.2021,750.0,PLN,,2/01/2022,12963,JAKUB GAWLICZEK,GRZEGORZ KACZAN,2022-01-10,,2022-01-10,2022-01-10,2022-01-10
135,8012,850.0,PLN,,13006,00046/FV/01/2022/MS,PT/2022/162-11.01/11.01.2022,750.0,PLN,,2/01/2022,12963,JAKUB GAWLICZEK,GRZEGORZ KACZAN,2022-01-11,,2022-01-11,2022-01-11,2022-01-11
180,8066,1470.0,PLN,,13049,00050/FV/01/2022/MS,85 /TD/01/2022,1420.0,PLN,7/2022,,,ARTUR FRĽSZCZAK,PAWEŁ MOLSKI,2022-01-13,,2022-01-14,2022-01-14,2022-01-14
444,8402,1020.0,PLN,,13583,00180/FV/01/2022/MS,002975,850.0,PLN,2022010215,,,ARTUR FRĽSZCZAK,NATALIA KANAS,2022-01-26,2022-01-26 00:00:00,2022-01-26,2022-01-26,2022-01-26
571,8746,2350.0,EUR,00023/FV/02/2022/MS,,,2022-01-22,2150.0,EUR,F-WAW-22-01280,,,ARTUR FRĽSZCZAK,MACIEJ SZUBA,2022-01-22,,2022-03-20,2022-02-17,2022-02-17
575,8807,2071.9,PLN,00001/KOR/03/2022/MS,,,SOLÓWKA,0.0,PLN,,,,ARTUR FRĽSZCZAK,ARTUR FRĽSZCZAK,2022-01-17,,2022-01-21,2022-01-21,2022-01-21
130,7999,850.0,EUR,,13128,00094/FV/01/2022/MS,533,650.0,EUR,30/T/22,,,ARTUR FRĽSZCZAK,MARCIN NIEDZWIEDZ,2022-01-10,,2022-01-11,2022-01-11,2022-01-11


In [ ]:
zlec_df.loc[:, 'ZA_DATA_RZ'] = zlec_df.ZA_DATA_RZ.map(lambda x: x[0])

In [ ]:
zlec_df.ZA_DATA_RZ.value_counts()

In [ ]:
zlec_df[data_cols].loc[zlec_df.ZA_DATA_RZ != 'NaN'].sample(3)

In [ ]:
zlec_df[zlec_cols].loc[zlec_df.FAKTURA_K_ZB_ID != ''].set_index('ID_ZLECENIA')

In [ ]:
relevant_zlec_ids = str(set(zlec_df.ID_ZLECENIA.to_list()))
relevant_zlec_ids = relevant_zlec_ids.replace('{', '(').replace('}', ')')


faktury_query = f"""
  SELECT * FROM FAKTURY WHERE ZLECENIE_ID IN {relevant_zlec_ids}"""

all_faktury_df = dataframe_from_query(cursor, faktury_query)

In [ ]:
investigated_cols = [el for el in all_faktury_df.columns if 'ZLEC' in el]
investigated_cols

In [ ]:
relevant_zlec_ids = str(set(zlec_df.NR_ZLECENIA_K.to_list()))
relevant_zlec_ids = relevant_zlec_ids.replace('{', '(').replace('}', ')')

faktury_query = f"""SELECT * FROM FAKTURY WHERE NR_ZLEC_K IN {relevant_zlec_ids}"""

all_faktury_df = dataframe_from_query(cursor, faktury_query)

In [ ]:
conn.close()